In [ ]:
project_dir = "/Users/joannarownicka/software/vad"
os.chdir(project_dir)

In [1]:
import glob
import sox
import json
import numpy as np
import os
from pathlib import Path

import matplotlib.pyplot as plt
import torch
from lhotse import LilcomFilesWriter

from lhotse.features import Fbank, FeatureSetBuilder
from lhotse.cut import CutSet, SupervisionSet
from lhotse.dataset.sampling import SingleCutSampler
from lhotse.dataset.vad import VadDataset

from torch.utils.data import DataLoader

from preprocessing import prepare_vad_dataset
from models import dnn

ModuleNotFoundError: No module named 'preprocessing'

In [ ]:
root_dir = Path('evaluation/data')
corpus_dir = root_dir / 'vad_data/'
output_dir = root_dir / 'vad_data_nb/'

In [ ]:
#vad_manifests = prepare_vad_dataset.prepare_vad_dataset(corpus_dir, output_dir)
cuts = CutSet.from_json(output_dir / 'cuts.json.gz')
cuts.describe()

In [ ]:
# Shuffle data but keep seed fixed, split into 80/10/10
#cuts_train, cuts_dev_eval = train_test_split(cuts, train_size=0.8, random_state=0)
#cuts_dev, cuts_eval = train_test_split(cuts_dev_eval, train_size=0.5, random_state=0)

vad_manifests = prepare_vad_dataset.prepare_vad_dataset(corpus_dir, output_dir)

train_ratio = 0.8

num_total = len(vad_manifests["supervisions"])
stop_train_idx = int(np.floor(num_total * train_ratio))
stop_dev_idx = int((num_total - stop_train_idx) // 2 + stop_train_idx)

train_ids, dev_ids, eval_ids = [], [], []
counter = 0
for sup_seg in vad_manifests["supervisions"]:
    id = sup_seg.to_dict()["id"]
    if counter < stop_train_idx:
        train_ids.append(id)
    elif counter < stop_dev_idx:
        dev_ids.append(id)
    else:
        eval_ids.append(id)
    counter += 1

assert train_ids[-1] != dev_ids[0]
assert dev_ids[-1] != eval_ids[0]

cuts_train = cuts.subset(supervision_ids=train_ids)
cuts_dev = cuts.subset(supervision_ids=dev_ids)
cuts_eval = cuts.subset(supervision_ids=eval_ids)

cuts_train.describe()
cuts_dev.describe()
cuts_eval.describe()


In [ ]:
vad_dataset = VadDataset()

train_sampler = SingleCutSampler(cuts_train.cut_into_windows(5.0, keep_excessive_supervisions=True), shuffle=False)
dev_sampler = SingleCutSampler(cuts_dev.cut_into_windows(5.0, keep_excessive_supervisions=True), shuffle=False)
eval_sampler = SingleCutSampler(cuts_eval.cut_into_windows(5.0, keep_excessive_supervisions=True), shuffle=False)

train_dloader = DataLoader(vad_dataset, sampler=train_sampler, batch_size=None, num_workers=1)
dev_dloader = DataLoader(vad_dataset, sampler=dev_sampler, batch_size=None, num_workers=1)
eval_dloader = DataLoader(vad_dataset, sampler=eval_sampler, batch_size=None, num_workers=1)

In [ ]:
input_size = 40 * 500

model = dnn(input_size=input_size, hidden_size=512, num_classes=2)

optim = torch.optim.Adam(model.parameters())

train_acc = []
valid_acc = []
for epoch in range(10):
  # training
    acc = []
    model.train()
    train_dloader.sampler.set_epoch(epoch)
    for batch_idx, data in enumerate(train_dloader):
        loss = model(*[d for d in data.values()])
        if batch_idx % 10 == 0:
            print(f'Batch {batch_idx} => loss {loss}')
        optim.zero_grad()
        loss.backward()
        acc.append(model.acc)
        norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        optim.step()
        t_r = loss.item()
    train_acc.append(np.mean(acc))
    # validation
    acc = []
    model.eval()
    for data in dev_dloader:
        model(*[d for d in data.values()])
        acc.append(model.acc)
    valid_acc.append(np.mean(acc))
    print(f"epoch: {epoch}, train acc: {train_acc[-1]:.3f}, dev acc: {valid_acc[-1]:.3f}, loss:{t_r:.3f}")

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()